In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import shared

ve_or_ju = "ju"

In [2]:
running_order = pd.read_csv(f'data/running_order_j2019_{ve_or_ju}.tsv', delimiter="\t")

In [3]:
running_order["leg_nro"] = running_order["leg"]
running_order["orig_name"] = running_order["name"]
running_order["name"] = running_order["name"].str.lower()

In [4]:
running_order.head()

,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name
0,28,OLV Baselland 1,OLV Baselland,SUI,1,10.9,timon schweizer,1,Timon Schweizer
1,28,OLV Baselland 1,OLV Baselland,SUI,2,10.5,kaspar hägler,2,Kaspar Hägler
2,28,OLV Baselland 1,OLV Baselland,SUI,3,13.2,fabian hertner,3,Fabian Hertner
3,28,OLV Baselland 1,OLV Baselland,SUI,4,7.3,jannis schönleber,4,Jannis Schönleber
4,28,OLV Baselland 1,OLV Baselland,SUI,5,7.8,stefan lombriser,5,Stefan Lombriser


In [5]:
predictions_and_history = pd.read_csv(f"data/preds_for_runners_with_history_2_{ve_or_ju}.csv", delimiter="\t")
predictions_and_history.head()


,Unnamed: 0,mean_team_id,num_valid_times,mean_pace,stdev,prior_mean,prior_log_std,predicted_pace,pred_pace_error,predicted_log_pace_mean,predicted_log_pace_std,name,teams
0,0,18.5,5,5.905,0.177,6.373,0.069,6.333,0.428,1.846,0.066,joni hirvikallio,KOOVEE
1,1,12.0,7,6.051,0.483,6.096,0.043,6.092,0.041,1.807,0.041,topi anjala,KOOVEE
2,2,3.0,7,5.933,0.382,6.244,0.068,6.195,0.262,1.824,0.063,lauri sild,DELTA;KOOVEE;HIIDENKIERTÄJÄT
3,3,18.0,3,6.069,0.363,6.431,0.119,6.396,0.327,1.856,0.114,oleksandr kratov,OK ORION;KOOVEE
4,4,21.0,5,5.936,0.345,6.080,0.071,6.075,0.139,1.804,0.067,kenny kivikas,KANGASALA SK;KOOVEE


In [6]:

predictions_and_history["num_runs"] = predictions_and_history["num_valid_times"]
no_history_row = pd.DataFrame([[0, 0, 0]], columns=["predicted_log_pace_mean", "predicted_log_pace_std", "num_valid_times"])
def get_history_and_preds(running_order_row):
    history_row = shared.get_matching_history_row_for_runner(running_order_row, predictions_and_history, no_history_row)
    #print(f"estimate_row log_means {history_row.log_means} {history_row.log_stdevs}")
    pred_log_mean = history_row.predicted_log_pace_mean.values[0]
    pred_log_std = history_row.predicted_log_pace_std.values[0]
    num_valid_times = history_row.num_valid_times.values[0]
    return pd.Series({"pred_log_mean": pred_log_mean, "pred_log_std": pred_log_std, "num_valid_times": num_valid_times})

history_and_preds = running_order.apply(lambda row: get_history_and_preds(row), axis=1)
running_order = running_order.assign(num_runs = history_and_preds.num_valid_times)
running_order = running_order.assign(pred_log_mean = history_and_preds.pred_log_mean)
running_order = running_order.assign(pred_log_std = history_and_preds.pred_log_std)



name 'juho siikilä' team_name 'KARJALAN PRIKAATI'
by_name 0 by_name_and_colon 2 runners 2
Duplicate runner       Unnamed: 0  mean_team_id  num_valid_times  mean_pace  stdev  prior_mean  \
5648        5683         912.5                4      8.399  0.524       9.651   
5649        5684        1115.0                1      9.864  0.000      10.674   

      prior_log_std  predicted_pace  pred_pace_error  predicted_log_pace_mean  \
5648          0.135           9.481            1.082                    2.249   
5649          0.163          10.649            0.785                    2.365   

      predicted_log_pace_std                                    name  \
5648                   0.126          juho siikilä:KARJALAN PRIKAATI   
5649                   0.160  juho siikilä:KARJALAN PRIKAATI (KARTR)   

                          teams  num_runs  
5648          KARJALAN PRIKAATI         4  
5649  KARJALAN PRIKAATI (KARTR)         1  
name 'petri laaksonen' team_name 'UNO SK'
by_name 0 by_n

In [7]:

(top_countries, top_first_names) = shared.read_persisted_dummy_column_values(ve_or_ju)

In [8]:
features = shared.preprocess_features(running_order, top_countries, ve_or_ju)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3253 entries, 0 to 3252
Data columns (total 12 columns):
team_id           3253 non-null int64
team              3253 non-null object
team_base_name    3253 non-null object
team_country      3253 non-null object
leg               3253 non-null int64
leg_dist          3253 non-null float64
name              3253 non-null object
leg_nro           3253 non-null int64
orig_name         3253 non-null object
num_runs          3253 non-null float64
pred_log_mean     3253 non-null float64
pred_log_std      3253 non-null float64
dtypes: float64(4), int64(3), object(5)
memory usage: 305.0+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1587 entries, 0 to 1586
Data columns (total 3 columns):
first_name           1586 non-null object
fn_pace_class        1587 non-null int64
fn_pace_std_class    1587 non-null int64
dtypes: int64(2), object(1)
memory usage: 37.3+ KB


None

['c_LTU', 'c_DEN', 'c_FRA']

In [9]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3253 entries, 0 to 3252
Data columns (total 47 columns):
team_id                  -3253 non-null int64
team_id_log10            -3253 non-null float64
team_id_square           -3253 non-null int64
leg_1                    -3253 non-null Sparse[uint8, 0]
leg_2                    -3253 non-null Sparse[uint8, 0]
leg_3                    -3253 non-null Sparse[uint8, 0]
leg_4                    -3253 non-null Sparse[uint8, 0]
leg_5                    -3253 non-null Sparse[uint8, 0]
leg_6                    -3253 non-null Sparse[uint8, 0]
leg_7                    -3253 non-null Sparse[uint8, 0]
c_EST                    -3253 non-null Sparse[uint8, 0]
c_FIN                    -3253 non-null Sparse[uint8, 0]
c_GBR                    -3253 non-null Sparse[uint8, 0]
c_GER                    -3253 non-null Sparse[uint8, 0]
c_LAT                    -3253 non-null Sparse[uint8, 0]
c_NOR                    -3253 non-null Sparse[uint8, 0]
c_OTHER     

In [10]:

#gbr_sd_estimate = shared.predict_without_history(features)
gbr_sd_estimate = shared.predict_without_history(features, ve_or_ju)

,log_q_low,predicted,log_q_high,log_std
0,1.927,4.281,1.960,0.017
1,1.776,6.382,1.915,0.069
2,1.782,6.348,1.981,0.100
3,1.908,6.252,2.094,0.093
4,1.827,6.963,2.039,0.106
5,1.908,4.419,1.948,0.020
6,1.759,5.420,1.858,0.050
7,1.785,7.290,2.008,0.112
8,1.820,7.436,2.038,0.109
9,1.838,7.520,2.031,0.097


0.1261649170019929

In [11]:
running_order["predicted"] = gbr_sd_estimate["predicted"]
running_order["log_q_low"] = gbr_sd_estimate["log_q_low"]
running_order["log_q_high"] = gbr_sd_estimate["log_q_high"]
running_order["log_std"] = gbr_sd_estimate["log_std"]


In [12]:
running_order.head()

,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std
0,28,OLV Baselland 1,OLV Baselland,SUI,1,10.9,timon schweizer,1,Timon Schweizer,4.0,1.935,0.129,4.281278,1.927112,1.960336,0.016612
1,28,OLV Baselland 1,OLV Baselland,SUI,2,10.5,kaspar hägler,2,Kaspar Hägler,2.0,1.842,0.051,6.382389,1.775996,1.914535,0.069270
2,28,OLV Baselland 1,OLV Baselland,SUI,3,13.2,fabian hertner,3,Fabian Hertner,6.0,1.746,0.070,6.347555,1.781784,1.980862,0.099539
3,28,OLV Baselland 1,OLV Baselland,SUI,4,7.3,jannis schönleber,4,Jannis Schönleber,2.0,1.949,0.118,6.251983,1.908036,2.094361,0.093162
4,28,OLV Baselland 1,OLV Baselland,SUI,5,7.8,stefan lombriser,5,Stefan Lombriser,4.0,2.007,0.110,6.963056,1.826875,2.038693,0.105909


In [13]:
running_order["log_std"].describe(percentiles=[0.01, 0.05, .25, .5, .75, .95, .99])

count    3253.000000
mean        0.126165
std         0.048764
min        -0.106473
1%         -0.027805
5%          0.043515
25%         0.104472
50%         0.126699
75%         0.151312
95%         0.201327
99%         0.259648
max         0.438145
Name: log_std, dtype: float64

In [14]:
running_order["log_std_fixed"] = np.clip(running_order["log_std"], 0.1, 0.5)
#running_order["log_std"].values[running_order["log_std"].values < 0] = 0.1

In [15]:
#def select_final_ind_preds(row):
#    return pd.Series({"pred_log_mean": pred_log_mean, "pred_log_std": pred_log_std, "num_valid_times": num_valid_times})
    
    
#final_ind_preds = running_order.apply(lambda row: select_final_ind_preds(row), axis=1)

running_order["final_pace_mean"] = np.log(running_order["predicted"])
running_order["final_pace_std"] = running_order["log_std_fixed"]
use_predicted_mean = running_order["num_runs"].values >= 1
running_order["final_pace_mean"].values[use_predicted_mean] = running_order["pred_log_mean"].values[use_predicted_mean]
use_predicted_std = running_order["num_runs"].values >= 3
running_order["final_pace_std"].values[use_predicted_std] = running_order["pred_log_std"].values[use_predicted_std]
running_order.head().round(3)

,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std,log_std_fixed,final_pace_mean,final_pace_std
0,28,OLV Baselland 1,OLV Baselland,SUI,1,10.9,timon schweizer,1,Timon Schweizer,4.0,1.935,0.129,4.281,1.927,1.960,0.017,0.100,1.935,0.129
1,28,OLV Baselland 1,OLV Baselland,SUI,2,10.5,kaspar hägler,2,Kaspar Hägler,2.0,1.842,0.051,6.382,1.776,1.915,0.069,0.100,1.842,0.100
2,28,OLV Baselland 1,OLV Baselland,SUI,3,13.2,fabian hertner,3,Fabian Hertner,6.0,1.746,0.070,6.348,1.782,1.981,0.100,0.100,1.746,0.070
3,28,OLV Baselland 1,OLV Baselland,SUI,4,7.3,jannis schönleber,4,Jannis Schönleber,2.0,1.949,0.118,6.252,1.908,2.094,0.093,0.100,1.949,0.100
4,28,OLV Baselland 1,OLV Baselland,SUI,5,7.8,stefan lombriser,5,Stefan Lombriser,4.0,2.007,0.110,6.963,1.827,2.039,0.106,0.106,2.007,0.110


In [16]:
#running_order.tail(15).round(3)

In [17]:
running_order.to_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", "\t")

In [18]:
running_order.tail().round(3)

,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std,log_std_fixed,final_pace_mean,final_pace_std
3248,1972,SEW Service 1,SEW Service,FIN,3,13.2,ville tuomala,3,Ville Tuomala,6.0,2.179,0.110,7.669,1.919,2.123,0.102,0.102,2.179,0.11
3249,1972,SEW Service 1,SEW Service,FIN,4,7.3,lauri ojansuu,4,Lauri Ojansuu,1.0,2.539,0.236,8.393,2.144,2.133,-0.005,0.100,2.539,0.10
3250,1972,SEW Service 1,SEW Service,FIN,5,7.8,jukka hatakka,5,Jukka Hatakka,0.0,0.000,0.000,11.689,2.312,2.228,-0.042,0.100,2.459,0.10
3251,1972,SEW Service 1,SEW Service,FIN,6,11.1,esko mattila,6,Esko Mattila,0.0,0.000,0.000,10.807,2.239,2.184,-0.028,0.100,2.380,0.10
3252,1972,SEW Service 1,SEW Service,FIN,7,12.9,toni anttonen,7,Toni Anttonen,0.0,0.000,0.000,7.703,1.996,2.038,0.021,0.100,2.042,0.10


In [19]:
running_order[
    ['num_runs', 'pred_log_mean', "pred_log_std", "predicted", "log_std_fixed", "final_pace_mean", "final_pace_std"]
].groupby('num_runs').agg(["mean"]).round(2)



,pred_log_mean,pred_log_std,predicted,log_std_fixed,final_pace_mean,final_pace_std
,mean,mean,mean,mean,mean,mean
num_runs,,,,,,
0.0,0.00,0.00,9.90,0.14,2.28,0.14
1.0,2.46,0.20,10.11,0.14,2.46,0.14
2.0,2.40,0.18,9.21,0.12,2.40,0.12
3.0,2.37,0.17,9.65,0.13,2.37,0.17
4.0,2.35,0.15,10.17,0.14,2.35,0.15
5.0,2.34,0.14,10.20,0.13,2.34,0.14
6.0,2.32,0.13,10.34,0.14,2.32,0.13
7.0,2.28,0.12,9.87,0.13,2.28,0.12
